In [11]:
# Convolutional Neural Network

# Importing the libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:
tf.__version__

'2.2.0'

In [13]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5327226957336777160
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 9834014616479322299
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3055235892
locality {
  bus_id: 1
  links {
  }
}
incarnation: 129341443862894867
physical_device_desc: "device: 0, name: GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 14698174248488940474
physical_device_desc: "device: XLA_GPU device"
]


In [14]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [15]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [16]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    r'D:\Python37\Projects\Nvidia_AI_Hackthon\dataset\training',
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
   r'D:\Python37\Projects\Nvidia_AI_Hackthon\dataset\training', # same directory as training data
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation') # set as validation data



# Preprocessing the Test set
#test_datagen = ImageDataGenerator(rescale = 1./255)

#test_generator  = test_datagen.flow_from_directory(r'D:\Python37\Projects\Nvidia_AI_Hackthon\dataset\test',
 #                                           target_size = (64, 64),
  #                                          batch_size = 32,
   #                                         class_mode = 'categorical')


Found 4700 images belonging to 2 classes.
Found 1175 images belonging to 2 classes.


In [74]:
# Part 2 - Building the CNN

# Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32,padding="same",kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=64,padding='same',kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))


# Adding a third convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=64,padding='same',kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a fourth convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=128,padding='same',kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=2, activation='softmax'))

In [75]:
cnn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 8, 8, 128)        

In [76]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint(r"D:\Python37\Projects\Nvidia_AI_Hackthon\models\model1.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 5,
                          verbose = 3,
                          restore_best_weights = True)

learning_rate= ReduceLROnPlateau(monitor = 'val_loss',
                                 patience = 3,
                                 verbose = 3)


# we put our call backs into a callback list
callbacks = [checkpoint, learning_rate]


In [83]:
# Part 3 - Training the CNN

# Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


In [84]:
# Training the CNN on the Training set and evaluating it on the Test set
history=cnn.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // 32,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples //32,
    callbacks=callbacks,
    epochs = 30)

Epoch 1/30
146/146 [==============================] - ETA: 0s - loss: 0.0806 - accuracy: 0.9717
Epoch 00001: val_loss did not improve from 0.10940
146/146 [==============================] - 9s 65ms/step - loss: 0.0806 - accuracy: 0.9717 - val_loss: 0.1378 - val_accuracy: 0.9514 - lr: 0.0010
Epoch 2/30
146/146 [==============================] - ETA: 0s - loss: 0.0765 - accuracy: 0.9754
Epoch 00002: val_loss did not improve from 0.10940
146/146 [==============================] - 9s 64ms/step - loss: 0.0765 - accuracy: 0.9754 - val_loss: 0.1329 - val_accuracy: 0.9557 - lr: 0.0010
Epoch 3/30
146/146 [==============================] - ETA: 0s - loss: 0.0708 - accuracy: 0.9730
Epoch 00003: val_loss improved from 0.10940 to 0.09749, saving model to D:\Python37\Projects\Nvidia_AI_Hackthon\models\model1.h5
146/146 [==============================] - 11s 77ms/step - loss: 0.0708 - accuracy: 0.9730 - val_loss: 0.0975 - val_accuracy: 0.9653 - lr: 0.0010
Epoch 4/30
146/146 [=========================

In [97]:
loss_tr, acc_tr = cnn.evaluate_generator(train_generator)

In [98]:
print('loss_training: ',loss_tr)
print('accuracy_training: ',acc_tr)

loss_training:  0.04264264926314354
accuracy_training:  0.9851064085960388


In [99]:
loss_val, acc_val= cnn.evaluate_generator(validation_generator)

In [100]:
print('loss_validation: ',loss_val)
print('accuracy_validation: ',acc_val)

loss_validation:  0.1069810763001442
accuracy_validation:  0.958297848701477


In [101]:
from tensorflow.keras.models import load_model

In [102]:
cnn= load_model(r'D:\Python37\Projects\Nvidia_AI_Hackthon\models\model1.h5')

In [103]:
import glob
from tensorflow.keras.preprocessing import image
import numpy as np
k = glob.glob(r"D:\Python37\Projects\Nvidia_AI_Hackthon\dataset\test\*.jpg")

y_pred = []
for i in k: 
    test_image = image.load_img(i, target_size = (64,64,3))
    test_image = image.img_to_array(test_image)
    test_image=test_image/255
    test_image = np.expand_dims(test_image, axis = 0)
    y_pred.append(cnn.predict(test_image))



In [104]:
labels=[]
for i in y_pred:
    labels.append(str(np.argmax(i[0])))

In [105]:
import json

j=[]
for i in range(1,100):
    j.append(str(i)+'.png')
    
res_dct3 = dict(zip(j, labels))


def write_json(filename, result):
    with open(filename, 'w') as outfile:
        json.dump(result, outfile)

write_json(r'D:\Python37\Projects\Nvidia_AI_Hackthon\result3.json',res_dct3)


         

In [106]:
import pandas as pd
pd.get_option("display.max_columns")
pd.set_option("display.max_rows", None)

In [107]:
pd.DataFrame(data=[res_dct,res_dct2,res_dct3],index=['res_dct1','res_dct2','res_dct3'])

,1.png,2.png,3.png,4.png,5.png,6.png,7.png,8.png,9.png,10.png,...,89.png,90.png,91.png,92.png,93.png,94.png,95.png,96.png,97.png,98.png
res_dct1,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,1,1,0,0,0,0
res_dct2,1,1,1,0,1,0,1,0,1,0,...,1,1,0,1,1,1,0,0,0,0
res_dct3,1,0,1,0,1,0,1,0,1,0,...,1,1,0,1,1,1,0,0,0,0
